In [5]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ, schedule
from qiskit.tools.jupyter import *
from qiskit.visualization import *
import time
from qiskit.providers.aer import QasmSimulator
import numpy as np
from qiskit_experiments.library import randomized_benchmarking as rb
#from qiskit_experiments.library import StandardRB, InterleavedRB
from qiskit_experiments.framework import ParallelExperiment, BatchExperiment
from qiskit_experiments.library.randomized_benchmarking import RBUtils
from qiskit_experiments.library import StandardRB, InterleavedRB
import qiskit.circuit.library as circuits
import qiskit.circuit.library
from qiskit.circuit import Delay, Instruction
from qiskit.circuit.library.standard_gates import XGate, SXGate, IGate, CCXGate, SGate
from datetime import datetime

from qiskit.circuit import Parameter
from qiskit_experiments.calibration_management.calibrations import Calibrations
# For simulation
from qiskit.providers.aer import AerSimulator

In [2]:
from qiskit import IBMQ
#IBMQ.save_account('befca44219528a564b7f4d3f53707de8985c22fb7b8c969ef2728c830de7c612716e76f41fbc0a52d65f63e71a3a2437948befd3042b1ffab4ced07d573192f1')
IBMQ.load_account()
IBMQ.providers()

[<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>,
 <AccountProvider for IBMQ(hub='ibm-q-community', group='qamp', project='spring-22-17')>,
 <AccountProvider for IBMQ(hub='ibm-q-community', group='qamp', project='spring-22-17-2')>,
 <AccountProvider for IBMQ(hub='ibm-q-skku', group='skku', project='squirl')>,
 <AccountProvider for IBMQ(hub='ibm-q-skku', group='skku', project='skku-graduate')>]

In [3]:
provider=IBMQ.providers()[3]
#provider = IBMQ.get_provider(hub='ibm-q-community', group='qamp', project='spring-22-17')
#backend=provider.backend.ibmq_lima
backend = provider.get_backend('ibmq_belem')

# get the basic feautures including qubit number of the backend.
config = backend.configuration()
backend_job_limit=backend.job_limit()

print("{0} is on version {1}. It has {2} qubit{3}. It "
      "{4} OpenPulse programs. The basis gates supported on this device are {5}.{6} are the list of connected qubits."
      "{7} can handle maximum {8} circuits(experiments) per job."
      "Maximum number of active job i can have on it (cocurrent jobs that a user is able to submit to a backend) is {9}."
      "".format(config.backend_name,
                config.backend_version,
                config.n_qubits,
                '' if config.n_qubits == 1 else 's',
                'supports' if config.open_pulse else 'does not support',
                config.basis_gates,
                config.coupling_map,
               config.backend_name,
               config.max_experiments,
               backend_job_limit.maximum_jobs))

ibmq_belem is on version 1.0.50. It has 5 qubits. It does not support OpenPulse programs. The basis gates supported on this device are ['id', 'rz', 'sx', 'x', 'cx', 'reset'].[[0, 1], [1, 0], [1, 2], [1, 3], [2, 1], [3, 1], [3, 4], [4, 3]] are the list of connected qubits.ibmq_belem can handle maximum 300 circuits(experiments) per job.Maximum number of active job i can have on it (cocurrent jobs that a user is able to submit to a backend) is None.


In [6]:
cals = Calibrations.from_backend(backend)

In [17]:
cals.get_schedule('xp',(0,))

CalibrationError: 'Schedule xp is not defined for qubits (0,).'

In [16]:
print(cals)

In [12]:
cals.add_parameter_value(5,'drive_freq')

In [13]:
cals.parameters


defaultdict(set,
            {Parameter(drive_freq): {ParameterKey(parameter='drive_freq', qubits=(), schedule=None)},
             Parameter(meas_freq): {ParameterKey(parameter='meas_freq', qubits=(), schedule=None)}})

In [15]:
print(backend.defaults().instruction_schedule_map.get('x',(1,)).instructions[0][1])

Play(Drag(duration=160, amp=(0.24659803755505416+0j), sigma=40, beta=0.6488775008745321, name='Xp_d1'), DriveChannel(1), name='Xp_d1')


In [16]:
print(backend.defaults().instruction_schedule_map.get('x',(1,)).instructions[0][1].pulse)

Drag(duration=160, amp=(0.24659803755505416+0j), sigma=40, beta=0.6488775008745321, name='Xp_d1')


In [14]:
print(backend.defaults().instruction_schedule_map.get('x',(1,)).instructions[0])

(0, Play(Drag(duration=160, amp=(0.24659803755505416+0j), sigma=40, beta=0.6488775008745321, name='Xp_d1'), DriveChannel(1), name='Xp_d1'))


In [22]:
print(backend.defaults().instruction_schedule_map.get('sx', (0,)))

Schedule((0, Play(Drag(duration=160, amp=(0.12338384889872352+0.005487369613793507j), sigma=40, beta=-2.459202708917089, name='X90p_d0'), DriveChannel(0), name='X90p_d0')), name="sx")


In [23]:
print(backend.defaults().instruction_schedule_map.get('x', (0,)))

Schedule((0, Play(Drag(duration=160, amp=(0.2556080519038154+0j), sigma=40, beta=-2.4187011998773236, name='Xp_d0'), DriveChannel(0), name='Xp_d0')), name="x")


In [24]:
print(backend.defaults().instruction_schedule_map.get('id', (0,)))

Schedule((0, Play(Waveform(array([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+

In [25]:
print(backend.defaults().instruction_schedule_map.get('cx', (0,1)))

Schedule((0, ShiftPhase(-3.141592653589793, DriveChannel(0))), (0, ShiftPhase(-1.5707963267948966, DriveChannel(1))), (0, ShiftPhase(-1.5707963267948966, ControlChannel(0))), (0, ShiftPhase(-3.141592653589793, ControlChannel(1))), (0, ShiftPhase(-1.5707963267948966, ControlChannel(4))), (0, ShiftPhase(-1.5707963267948966, ControlChannel(5))), (0, Play(Drag(duration=160, amp=(-0.00548736961379351+0.12338384889872352j), sigma=40, beta=-2.459202708917089, name='Y90p_d0'), DriveChannel(0), name='Y90p_d0')), (0, Play(Drag(duration=160, amp=(0.11935921573597799+0.0023128406808406456j), sigma=40, beta=0.6643204672670971, name='X90p_d1'), DriveChannel(1), name='X90p_d1')), (160, Play(GaussianSquare(duration=1584, amp=(0.022135754720986243+0.004321499197552103j), sigma=64, width=1328, name='CR90p_d0_u1'), DriveChannel(0), name='CR90p_d0_u1')), (160, Play(GaussianSquare(duration=1584, amp=(-0.1369366200673462+0.047374757126980735j), sigma=64, width=1328, name='CR90p_u1'), ControlChannel(1), name

In [5]:
print(backend.defaults())

<PulseDefaults(<InstructionScheduleMap(1Q instructions:
  q0: {'sx', 'u2', 'u3', 'id', 'rz', 'x', 'u1', 'measure'}
  q1: {'sx', 'u2', 'u3', 'id', 'rz', 'x', 'u1', 'measure'}
  q2: {'sx', 'u2', 'u3', 'id', 'rz', 'x', 'u1', 'measure'}
  q3: {'sx', 'u2', 'u3', 'id', 'rz', 'x', 'u1', 'measure'}
  q4: {'sx', 'u2', 'u3', 'id', 'rz', 'x', 'u1', 'measure'}
Multi qubit instructions:
  (0, 1): {'cx'}
  (1, 0): {'cx'}
  (1, 2): {'cx'}
  (1, 3): {'cx'}
  (2, 1): {'cx'}
  (3, 1): {'cx'}
  (0, 1, 2, 3, 4): {'measure'}
)>Qubit Frequencies [GHz]
[5.090215187962305, 5.245528840279351, 5.361492855859864, 5.1703881423849465, 5.254553035940048]
Measurement Frequencies [GHz]
[7.301574705, 7.393433372, 7.360065181, 7.303382327, 7.426223338] )>
